# **INTRODUCTIONS**

---

The code is separated into multiple sections:

1. **[Tables](#tables)**

# **DEPENDENCIES**

---

```json
{
    "depends-on": [
        "src/utils/schemas.ipynb",
        "src/statics/reclassified_municipality_mapping.ipynb"
    ]
}
```

# **TABLES**

---

### STAGES

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE OBJECT_ID = OBJECT_ID(N'[DBO].[SEQ_VWSSTAGE_CBS_POPULATION_BASE]') AND TYPE = 'SO')
CREATE SEQUENCE SEQ_VWSSTAGE_CBS_POPULATION_BASE
  START WITH 1
  INCREMENT BY 1;
GO

-- 2) CREATE STAGE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTAGE].[CBS_POPULATION_BASE]'))
CREATE TABLE [VWSSTAGE].[CBS_POPULATION_BASE](
	[ID] INT PRIMARY KEY NONCLUSTERED NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTAGE_CBS_POPULATION_BASE]),
	[DATE_LAST_INSERTED] DATETIME DEFAULT GETDATE(),
  [GEMEENTE_CODE]  VARCHAR(100) NULL,
  [GEMEENTE] VARCHAR(100) NULL,
  [LEEFTIJD] VARCHAR(100) NULL,
  [GESLACHT] VARCHAR(100) NULL,
  [DATUM_PEILING] VARCHAR(100) NULL,
  [POPULATIE] VARCHAR(100) NULL,
  [VEILIGHEIDSREGIO_CODE] VARCHAR(100) NULL,
  [VEILIGHEIDSREGIO_NAAM] VARCHAR(100) NULL,
  [PROVINCIE_CODE] VARCHAR(100) NULL,
  [PROVINCIE_NAAM] VARCHAR(100) NULL,
  [GGD_CODE] VARCHAR(100) NULL,
  [GGD_NAAM] VARCHAR(100) NULL
);
GO

-- 3) CREATE STAGE INDEXE(S).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'CI_DLI_VWSSTAGE_CBS_POPULATION_BASE')
CREATE CLUSTERED INDEX [CI_DLI_VWSSTAGE_CBS_POPULATION_BASE] ON [VWSSTAGE].[CBS_POPULATION_BASE] (
  [DATE_LAST_INSERTED]
);
GO

IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'NCI_DLI_VWSSTAGE_CIMS_VACCINATED_AGE_GROUP')
CREATE NONCLUSTERED INDEX [NCI_DLI_VWSSTAGE_CIMS_VACCINATED_AGE_GROUP] ON [VWSSTAGE].[CBS_POPULATION_BASE] (
  [DATE_LAST_INSERTED], 
  [GEMEENTE_CODE], 
  [GEMEENTE], 
  [LEEFTIJD], 
  [GESLACHT], 
  [DATUM_PEILING], 
  [POPULATIE], 
  [VEILIGHEIDSREGIO_CODE], 
  [VEILIGHEIDSREGIO_NAAM], 
  [PROVINCIE_CODE], 
  [PROVINCIE_NAAM], 
  [GGD_CODE], 
  [GGD_NAAM]
) WITH (STATISTICS_NORECOMPUTE = OFF, DROP_EXISTING = OFF, ONLINE = OFF, OPTIMIZE_FOR_SEQUENTIAL_KEY = OFF) ON [PRIMARY];
GO


### STATICS

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATION FOR MORE INFORMATION.

SET ANSI_NULLS ON
GO

SET QUOTED_IDENTIFIER ON
GO

-- 1) CREATE STAGE SEQUENCE(S).....
IF NOT EXISTS(SELECT * FROM SYS.SEQUENCES WHERE [OBJECT_ID] = OBJECT_ID(N'[DBO].[SEQ_VWSSTATIC_CBS_POPULATION_BASE]') AND [TYPE] = 'SO')
CREATE SEQUENCE [SEQ_VWSSTATIC_CBS_POPULATION_BASE]
	START WITH 1
	INCREMENT BY 1;
GO

-- 2) CREATE STAGE TABLE(S).....
IF NOT EXISTS(SELECT * FROM SYS.TABLES WHERE [OBJECT_ID] = OBJECT_ID(N'[VWSSTATIC].[CBS_POPULATION_BASE]'))
CREATE TABLE [VWSSTATIC].[CBS_POPULATION_BASE](
	[ID] [INT] PRIMARY KEY NONCLUSTERED NOT NULL DEFAULT (NEXT VALUE FOR [DBO].[SEQ_VWSSTATIC_CBS_POPULATION_BASE]),
	[GEMEENTE_CODE]  [VARCHAR](10) NULL,
	[GEMEENTE] [VARCHAR](100) NULL,
	[LEEFTIJD] [INT] NULL,
	[GESLACHT] [VARCHAR](10) NULL,
	[DATUM_PEILING] [DATE] NULL,
	[POPULATIE] [INT] NULL,
	[VEILIGHEIDSREGIO_CODE] [VARCHAR](10) NULL,
	[VEILIGHEIDSREGIO_NAAM] [VARCHAR](100) NULL,
	[PROVINCIE_CODE] [VARCHAR](10) NULL,
	[PROVINCIE_NAAM] [VARCHAR](100) NULL,
	[GGD_CODE] [VARCHAR](10) NULL,
	[GGD_NAAM] [VARCHAR](100) NULL,
	[DATE_LAST_INSERTED] [DATETIME] DEFAULT GETDATE()
);
GO

-- 3) CREATE STAGE INDEXE(S).....
IF NOT EXISTS (SELECT * FROM SYS.INDEXES WHERE [NAME] = 'CI_DLI_VWSSTATIC_CBS_POPULATION_BASE')
CREATE CLUSTERED INDEX [CI_DLI_VWSSTATIC_CBS_POPULATION_BASE] ON [VWSSTATIC].[CBS_POPULATION_BASE] (
    [DATE_LAST_INSERTED] ASC
);
GO


# **STORED PROCEDURES**

---

In 2022 multiple municipalties (i.e. gemeentes) are reclassified (e.g. merging) into novel muncipalities. To implement these changes, several database adjustments had to be performed.

Within the `[DBO].[SP_CBS_POPULATION_BASE_STATIC]`, with help of the mapping-tabel (i.e. `[RECLASSIFIED_MUNICIPALITY_MAPPING]` ), the reclassified muncipalities will be progated towards the `[VWSSTATIC].[CBS_POPULATION_BASE]`:

```sql
LEFT JOIN (
    SELECT 
        * 
    FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
    WHERE [ACTIVE] = 1 
        AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
) B ON A.[GEMEENTE_CODE]=B.[GM_CODE_ORIGINAL]
```

In turn, all downstream procedures, views or tabel - depending on `[VWSSTATIC].[CBS_POPULATION_BASE]` - will automatcally implement the modified muncipality codes (i.e. `GM_CODE`, `GMCODE` or `GEMEENTE_CODE`).

### STAGING &rarr; STATIC MAPPING

In [ ]:
-- COPYRIGHT (C) 2020 DE STAAT DER NEDERLANDEN, MINISTERIE VAN VOLKSGEZONDHEID, WELZIJN EN SPORT.
-- LICENSED UNDER THE EUROPEAN UNION PUBLIC LICENCE V. 1.2 - SEE HTTPS://GITHUB.COM/MINVWS/NL-CONTACT-TRACING-APP-COORDINATIONFOR MORE INFORMATION.

CREATE OR ALTER PROCEDURE [DBO].[SP_CBS_POPULATION_BASE_STATIC]
AS
BEGIN

-- POPULATION_BASE IS TRANSFERRED FROM STAGE TO STATIC, THE TABLE RECODE_MUNICIPALITY IS USED TO APPLY CHANGES IN THE MUNICIPALITIES.
-- THE RECODE_MUNICIPALITY TABLE IS LEFT-JOINED, SO ONLY MUNICIPALITIES THAT MET WITH A CHANGE WILL BE AFFECTED. 
-- ONLY CHANGES THAT ARE 'ACTIVE' ARE SELECTED.
INSERT INTO [VWSSTATIC].[CBS_POPULATION_BASE] (
    [GEMEENTE_CODE]
    ,[GEMEENTE]
    ,[LEEFTIJD]
    ,[GESLACHT]
    ,[DATUM_PEILING]
    ,[POPULATIE]
    ,[VEILIGHEIDSREGIO_CODE]
    ,[VEILIGHEIDSREGIO_NAAM]
    ,[PROVINCIE_CODE]
    ,[PROVINCIE_NAAM]
    ,[GGD_CODE]
    ,[GGD_NAAM]
)
SELECT 
    ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE]) AS [GEMEENTE_CODE]
    ,ISNULL(B.[GM_NAME_NEW],[GEMEENTE]) AS [GEMEENTE]
    ,A.[LEEFTIJD]
    ,A.[GESLACHT]
    ,CAST(A.[DATUM_PEILING] AS DATE) AS [DATUM_PEILING]
    ,ROUND(SUM(CAST(A.[POPULATIE] AS INT) * ISNULL(B.[SHARES],1.0)),0) AS [POPULATIE] 
    ,ISNULL(B.[VR_CODE],[VEILIGHEIDSREGIO_CODE]) AS [VEILIGHEIDSREGIO_CODE]
    ,ISNULL(B.[VR_NAME],[VEILIGHEIDSREGIO_NAAM]) AS [VEILIGHEIDSREGIO_NAAM]
    ,ISNULL(B.[PROVINCE_CODE],[PROVINCIE_CODE]) AS [PROVINCIE_CODE]
    ,ISNULL(B.[PROVINCE_NAME],[PROVINCIE_NAAM]) AS [PROVINCIE_NAAM]
    ,ISNULL(B.[GGD_CODE], A.[GGD_CODE]) AS [GGD_CODE]
    ,ISNULL(B.[GGD_NAME],[GGD_NAAM]) AS [GGD_NAAM]
FROM [VWSSTAGE].[CBS_POPULATION_BASE] A
    LEFT JOIN (
        SELECT 
            * 
        FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING] 
        WHERE [ACTIVE] = 1 
            AND [DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTATIC].[RECLASSIFIED_MUNICIPALITY_MAPPING])  
    ) B ON A.[GEMEENTE_CODE] = B.[GM_CODE_ORIGINAL]
WHERE A.[DATE_LAST_INSERTED] = (SELECT MAX([DATE_LAST_INSERTED]) FROM [VWSSTAGE].[CBS_POPULATION_BASE])
GROUP BY
    ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE]) 
    ,ISNULL(B.[GM_NAME_NEW],[GEMEENTE]) 
    ,A.[LEEFTIJD]
    ,A.[GESLACHT]
    ,CAST(A.[DATUM_PEILING] AS DATE) 
    ,ISNULL(B.[VR_CODE],[VEILIGHEIDSREGIO_CODE]) 
    ,ISNULL(B.[VR_NAME],[VEILIGHEIDSREGIO_NAAM]) 
    ,ISNULL(B.[PROVINCE_CODE],[PROVINCIE_CODE]) 
    ,ISNULL(B.[PROVINCE_NAME],[PROVINCIE_NAAM]) 
    ,ISNULL(B.[GGD_CODE],A.[GGD_CODE]) 
    ,ISNULL(B.[GGD_NAME],[GGD_NAAM]) 
ORDER BY [DATUM_PEILING], ISNULL(B.[GM_CODE_NEW],[GEMEENTE_CODE])
END;
GO